In [6]:
import numpy as np
import pandas as pd
import json

# machine learning
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
import lightgbm as lgb
from sklearn.metrics import explained_variance_score
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV

# ignore Warnings
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime

In [21]:
train_df = pd.read_csv("train_change.csv", encoding='euc-kr')

In [23]:
train_df.head()

,Unnamed: 0,분석데이터,label,numstrings,avlength,printables,entropy,paths,urls,registry,...,B247,B248,B249,B250,B251,B252,B253,B254,B255,B256
0,0,1,1,144,12.298611,1771,5.356616,0,0,0,...,14134,11491,14357,11245,12666,13568,10658,11016,10611,15287
1,1,2,1,804,9.580846,7703,6.063542,0,0,0,...,976,957,834,976,792,1063,796,932,804,1361
2,2,3,0,2205,12.736054,28083,6.107050,9,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,4,0,2602,10.288240,26770,5.373013,8,0,0,...,147,146,143,132,123,117,77,97,100,91
4,4,5,1,8980,23.252339,208806,5.775223,0,28,16,...,19717,20152,16210,20032,19894,19970,18737,20374,23125,19412


In [25]:
train_df_x = train_df.drop(['Unnamed: 0', '분석데이터','label'], axis=1)

In [27]:
train_df_y = train_df['label']

In [28]:
train_df_y = train_df_y.astype('int')

In [34]:
x_train, x_test, y_train, y_test = train_test_split(train_df_x, train_df_y, test_size=0.2, random_state=42)

In [36]:
x_train.astype('float')

,numstrings,avlength,printables,entropy,paths,urls,registry,MZ,printabledist1,printabledist2,...,B247,B248,B249,B250,B251,B252,B253,B254,B255,B256
9254,92.0,13.500000,1242.0,5.617271,0.0,0.0,0.0,1.0,14.0,3.0,...,116391.0,122848.0,114520.0,116821.0,115633.0,121718.0,117862.0,121434.0,122693.0,126309.0
1561,629.0,11.941176,7511.0,6.061898,0.0,0.0,0.0,4.0,252.0,21.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1670,362.0,5.748619,2081.0,6.047759,0.0,0.0,0.0,7.0,45.0,36.0,...,103661.0,132596.0,205883.0,77645.0,76928.0,87756.0,119757.0,95879.0,128903.0,146837.0
6087,27790.0,5.742965,159597.0,6.560686,0.0,0.0,0.0,75.0,1770.0,1394.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6669,2042.0,8.457884,17271.0,6.306899,0.0,9.0,0.0,3.0,754.0,105.0,...,303402.0,327390.0,300929.0,310182.0,308986.0,319403.0,313699.0,320150.0,327282.0,368379.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,728.0,21.414835,15590.0,5.339247,0.0,0.0,0.0,1.0,18.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5191,31774.0,11.639391,369830.0,5.401709,0.0,31.0,0.0,1.0,1230.0,374.0,...,26603.0,29099.0,24420.0,27278.0,26287.0,25216.0,26759.0,28166.0,33345.0,25990.0
5390,596.0,7.041946,4197.0,6.054880,0.0,0.0,0.0,6.0,31.0,17.0,...,269124.0,258899.0,284075.0,281231.0,266345.0,294071.0,319813.0,274277.0,252110.0,358873.0
860,205.0,11.965854,2453.0,5.786017,0.0,0.0,0.0,1.0,42.0,1.0,...,485237.0,523476.0,583697.0,406096.0,488099.0,446034.0,477358.0,456855.0,489706.0,472279.0


In [57]:
train_ds = lgb.Dataset(x_train, label = y_train)
test_ds = lgb.Dataset(x_test, label = y_test)

params = {'learning_rate':0.01,
         'max_depth':-1,
         'boosting':'gbdt',
         'objective': 'binary',
         'metric': 'binary_logloss',
         'is_training_metric': True,
         'num_leaves':144,
         'feature_fraction':0.9,
         'bagging_fraction':0.7,
         'bagging_freq':5,
         'seed':2020}

model = lgb.train(params, train_ds, 1000, test_ds, verbose_eval=100, early_stopping_rounds=100)
y_pred = model.predict(x_test)

[LightGBM] [Info] Number of positive: 4400, number of negative: 3600
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.045995 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 151567
[LightGBM] [Info] Number of data points in the train set: 8000, number of used features: 616
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.550000 -> initscore=0.200671
[LightGBM] [Info] Start training from score 0.200671
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.378118
[200]	valid_0's binary_logloss: 0.274241
[300]	valid_0's binary_logloss: 0.226715
[400]	valid_0's binary_logloss: 0.203107
[500]	valid_0's binary_logloss: 0.190899
[600]	valid_0's binary_logloss: 0.185068
[700]	valid_0's binary_logloss: 0.181481
[800]	valid_0's binary_logloss: 0.18089
Early stopping, best iteration is:
[755]	

In [58]:
for i in range(len(y_pred)):
    if y_pred[i]>=.5:
         y_pred[i]=1
    else:
        y_pred[i]=0

In [59]:
accuracy = accuracy_score(y_pred, y_test)

accuracy

0.9265